<a href="https://colab.research.google.com/github/viksingh/pythonfinance/blob/master/bollingerbandsstrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import glob
import os
import numpy as np
import sys
import time
import requests
from io import StringIO

start_time = time.time()

#!{sys.executable} -m pip install PyDrive 

from pydrive.auth import  GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Check if index is going up
MONEYFOREACHSTOCK = 2500
AMOUNTSPENT = 0
AMOUNTEARNERD = 0
SELLPRICE = 0
STOPLOSSFACTORINCREASINGINDEX = 0.80
STOPLOSSFACTORREDUCINGINDEX = 0.90
NUMBEROFSTOCKS = 0
CURRENTAMOUNT = 0
TOTALSTOCKSHELD = 0
STOCKVALUE = 0

TOTALSPENT = 0
TOTALVALUE = 0
TOTALSTOPLOSSMONEY = 0


filePathIndex = '/content/drive/My Drive/Colab Notebooks/data/index2.csv'
filePathCombined = '/content/drive/My Drive/Colab Notebooks/data/stockscombined.csv'

dfindex = pd.read_csv(filePathIndex, header=0)
dfstocks = pd.read_csv(filePathCombined, header=0)

# Create a dataframe to store currently bought stocks
# Fields : STOCK, BOUGHT DATE , BOUGHT PRICE , QTY, UPDATE DATE, LAST PRICE, RoC   , BOUGHTPRICE , UPDTPRICE
# May be add an entry for each week when this gets checked
print('READ BOTH FILES')

from datetime import date, timedelta

start_date = date(2019, 12, 20)
end_date = date(2019, 12, 27)
delta = timedelta(days=7)
dfStockUniqueList = dfstocks
# dfStockUniqueList[1].unique()
# dfStockUniqueList['STOCK'] = dfStockUniqueList.groupby('STOCK')['STOCK'].rank( ascending=False)
companiesList = dfStockUniqueList.STOCK.unique()
# print(companiesList)

dfStocksCurrentlyBought = pd.DataFrame(columns=['STOCK', 'PURCHASEDATE', 'PURCHASEPRICE', 'QTY', 'UPDATEDATE', 'UPDATEPRICE', 'MAXPRICE', 'ROC', 'ACTIVE'])

while start_date <= end_date:
    # Check if index is increasing / decreasing this week
    StartDate = start_date.strftime("%Y-%m-%d")
    queryIndexWithDate = 'DATE' + '==' + "'" + StartDate + "'"
    dfNasdaqCheckIndexIncreasing = dfindex.query(queryIndexWithDate)
    if dfNasdaqCheckIndexIncreasing.size > 0:
        isIncreasing = dfNasdaqCheckIndexIncreasing['CROSS'].iloc[-1]
        print('Index : ' + str(StartDate) + ' - ' + str(isIncreasing))

        # check if any stock needs to be sold in the current week
        # loop over stocks currently bought

        # ######## SELL STOCK LOGIC ###################################################
        # ######## SELL STOCK LOGIC ###################################################

        dfstocksActive = dfStocksCurrentlyBought.query("ACTIVE == 'X'")
        stockBoughtList = dfstocksActive.STOCK.unique()

        for stockToSellCheck in stockBoughtList:
            #                print('CHECK IF WE SHOULD SELL ')

            queryGetCurrentStockDataSell = 'STOCK' + '==' + "'" + stockToSellCheck + "'"
            dfCurrentStockSell = dfstocks.query(queryGetCurrentStockDataSell)
            queryGetCurrentPriceWithDateSell = "DATE" + "<=" + "'" + StartDate + "'"
            dfCurrentStockSell = dfCurrentStockSell.query(queryGetCurrentPriceWithDateSell)
            dfCurrentStockSell = dfCurrentStockSell.tail(2)

            if dfCurrentStockSell.shape[0] == 2:

                #                    print('Checking' + str(dfCurrentStockSell.size) + 'for stock : '+ str(stock))

                CURRENTSTOCKPRICESellTHISWEEK = dfCurrentStockSell['PRICE'].iloc[1]
                CURRENTSTOCKPRICEUPPERBOLLINGERTHISWEEK = dfCurrentStockSell['UPPERBOLLINGER'].iloc[1]
                CURRENTSTOCKPRICELOWERBOLLINGERTHISWEEK = dfCurrentStockSell['LOWERBOLLINGER'].iloc[1]

                CURRENTSTOCKPRICESellLASTWEEK = dfCurrentStockSell.loc[dfCurrentStockSell.index[0], 'PRICE']
                CURRENTSTOCKPRICEUPPERBOLLINGERLASTWEEK = dfCurrentStockSell.loc[dfCurrentStockSell.index[0], 'UPPERBOLLINGER']
                CURRENTSTOCKPRICELOWERBOLLINGERLASTWEEK = dfCurrentStockSell.loc[dfCurrentStockSell.index[0], 'LOWERBOLLINGER']

                if (CURRENTSTOCKPRICESellTHISWEEK < CURRENTSTOCKPRICELOWERBOLLINGERTHISWEEK and CURRENTSTOCKPRICESellLASTWEEK > CURRENTSTOCKPRICELOWERBOLLINGERLASTWEEK):
                    queryGetCurrentStockDataSell = 'STOCK' + '==' + "'" + stockToSellCheck + "'"
                    dfCurrentStockDataForQty = dfStocksCurrentlyBought.query(queryGetCurrentStockDataSell)
                    STOCKQTY = dfCurrentStockDataForQty.iloc[-1]['QTY']
                    print('Sell : @ ' + str(stockToSellCheck) + ' - ' + str(StartDate) + ' - ' + str(CURRENTSTOCKPRICESellTHISWEEK) + ' - ' + str(CURRENTSTOCKPRICESellTHISWEEK * STOCKQTY))
                    dfStocksCurrentlyBought.loc[(dfStocksCurrentlyBought.STOCK == stockToSellCheck), 'ACTIVE'] = ''
                    STOCKVALUELOSS = 0
                    STOCKVALUELOSS =  STOCKVALUELOSS + CURRENTSTOCKPRICESellTHISWEEK
                    TOTALVALUE = TOTALVALUE + STOCKVALUELOSS

                # Get current and previous week. Identify stocks

        # SELLING RULES
        # If increasing index this week : current price is higher than 0.8 MAX PRICE which will be the last row of
        # If currennt price

        # **************** IDENTIFY STOCKS TO BUY ***********************************
        # **************** IDENTIFY STOCKS TO BUY ***********************************

        if (isIncreasing == 1):
            for stockToBuy in companiesList:
                #                print("Analysing stock :  " + stock)

                queryGetCurrentStockDataBuy = 'STOCK' + '==' + "'" + stockToBuy + "'"
                dfCurrentStockBuy = dfstocks.query(queryGetCurrentStockDataBuy)
                queryGetCurrentPriceWithDateBuy = "DATE" + "<=" + "'" + StartDate + "'"
                dfCurrentStockBuy = dfCurrentStockBuy.query(queryGetCurrentPriceWithDateBuy)
                dfCurrentStockBuy = dfCurrentStockBuy.tail(2)


                if dfCurrentStockBuy.shape[0] == 2:

                    #                    print('Checking' + str(dfCurrentStockBuy.size) + 'for stock : '+ str(stock))

                    CURRENTSTOCKPRICEBUYTHISWEEK = dfCurrentStockBuy['PRICE'].iloc[1]
                    CURRENTSTOCKPRICEUPPERBOLLINGERTHISWEEK = dfCurrentStockBuy['UPPERBOLLINGER'].iloc[1]
                    CURRENTSTOCKPRICELOWERBOLLINGERTHISWEEK = dfCurrentStockBuy['LOWERBOLLINGER'].iloc[1]

                    CURRENTSTOCKPRICEBUYLASTWEEK = dfCurrentStockBuy.loc[dfCurrentStockBuy.index[0], 'PRICE']
                    CURRENTSTOCKPRICEUPPERBOLLINGERLASTWEEK = dfCurrentStockBuy.loc[dfCurrentStockBuy.index[0], 'UPPERBOLLINGER']
                    CURRENTSTOCKPRICELOWERBOLLINGERLASTWEEK = dfCurrentStockBuy.loc[dfCurrentStockBuy.index[0], 'LOWERBOLLINGER']

                    if (CURRENTSTOCKPRICEBUYTHISWEEK > CURRENTSTOCKPRICEUPPERBOLLINGERTHISWEEK and CURRENTSTOCKPRICEBUYLASTWEEK < CURRENTSTOCKPRICEUPPERBOLLINGERLASTWEEK and CURRENTSTOCKPRICEBUYTHISWEEK > 5):
                        NUMBEROFSTOCKS = int(MONEYFOREACHSTOCK / CURRENTSTOCKPRICEBUYTHISWEEK)

                        AMOUNTSPENTFORTHISSTOCK = NUMBEROFSTOCKS * CURRENTSTOCKPRICEBUYTHISWEEK
                        AMOUNTSPENT = AMOUNTSPENT + AMOUNTSPENTFORTHISSTOCK
                        print('Buy : ' + str(stockToBuy) + ' - ' + str(StartDate) + ' @ ' + str(CURRENTSTOCKPRICEBUYTHISWEEK) + ' - ' + str(AMOUNTSPENTFORTHISSTOCK))
                        dfStocksCurrentlyBought = dfStocksCurrentlyBought.append(
                            {'STOCK': stockToBuy, 'PURCHASEDATE': StartDate, 'PURCHASEPRICE': CURRENTSTOCKPRICEBUYTHISWEEK, 'QTY': NUMBEROFSTOCKS, 'UPDATEDATE': StartDate,
                             'UPDATEPRICE': CURRENTSTOCKPRICEBUYTHISWEEK, 'MAXPRICE': CURRENTSTOCKPRICEBUYTHISWEEK, 'ROC': 100, 'ACTIVE': 'X'}, ignore_index=True)


    else:
        print('NO RECOMMENDATIONS AS INDEX IS GOING DOWN : ' + str(start_date))

    EndDate = end_date.strftime("%Y-%m-%d")

    if (StartDate == EndDate):
        stockCurrtList = dfStocksCurrentlyBought.STOCK.unique()


        for stockCurrValue in stockCurrtList:
            queryGetCurrentStockDataCurrValue = 'STOCK' + '==' + "'" + stockCurrValue + "'"
            dfCurrentStockCurrValue = dfstocks.query(queryGetCurrentStockDataCurrValue)

            queryGetCurrentPriceWithDateCurrValue = "DATE" + "==" + "'" + EndDate + "'"
            dfCurrentStockCurrValue = dfCurrentStockCurrValue.query(queryGetCurrentPriceWithDateCurrValue)
            dfCurrentStockCurrValue = dfCurrentStockCurrValue.tail(1)

            if dfCurrentStockCurrValue.shape[0] > 0:
                PRICEFORSUMMARY = float(dfCurrentStockCurrValue['PRICE'].iloc[0])

                dfCurrentStockQty = dfStocksCurrentlyBought.query(queryGetCurrentStockDataCurrValue)['QTY'].iloc[0]
                STOCKVALUE = float(PRICEFORSUMMARY * dfCurrentStockQty)
                print('STOCK : ' + stockCurrValue + ' - ' + str(CURRENTSTOCKPRICEBUYTHISWEEK) + '- ' + str(dfCurrentStockQty) + ' - ' + str(STOCKVALUE))
                TOTALVALUE = TOTALVALUE + STOCKVALUE

                print(str(stockCurrValue) + ' - ' + str(dfCurrentStockQty) + ' - ' + str(EndDate) + ' - ' + str(STOCKVALUE))

        print('AMOUNT SPENT : ' + str(AMOUNTSPENT) + ' CURR VALUE: ' + str(TOTALVALUE))

    start_date += delta

READ BOTH FILES
<bound method NDFrame.tail of             DATE        PRICE           MA  CROSS
0     2001-01-02  2291.860107     0.000000      0
1     2001-01-03  2616.689941     0.000000      0
2     2001-01-04  2566.830078     0.000000      0
3     2001-01-05  2407.649902     0.000000      0
4     2001-01-08  2395.919922     0.000000      0
...          ...          ...          ...    ...
4772  2019-12-20  8924.959961  8469.920010      1
4773  2019-12-23  8945.650391  8487.692217      1
4774  2019-12-24  8952.879883  8505.776816      1
4775  2019-12-26  9022.389648  8523.250410      1
4776  2019-12-27  9006.620117  8540.899209      1

[4777 rows x 4 columns]>
<bound method NDFrame.head of          Unnamed: 0    PRICE        DATE  ...  STD UPPERBOLLINGER  LOWERBOLLINGER
0                 3  39.0000  2017-09-01  ...  NaN            NaN             NaN
1                 7  49.9300  2017-09-08  ...  NaN            NaN             NaN
2                12  24.1155  2017-09-15  ...  NaN  

In [0]:
from google.colab import drive
drive.mount('/content/drive')